<a href="https://colab.research.google.com/github/andrewhylee/Image-Recognizer/blob/master/CancerPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import time
import json
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import PIL

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from PIL import Image
from collections import OrderedDict
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

In [0]:
data_transforms = {    
    'train': transforms.Compose([        
                                 transforms.RandomRotation(30),        
                                 transforms.RandomResizedCrop(224),        
                                 transforms.RandomHorizontalFlip(),        
                                 transforms.ToTensor(),        
                                 transforms.Normalize([0.485, 0.456, 0.406],  [0.229, 0.224, 0.225])    ]
                                ),    
                  'valid': transforms.Compose([        
                                               transforms.Resize(256),        
                                               transforms.CenterCrop(224),        
                                               transforms.ToTensor(),        
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])    ]
                                              )
                  }


In [0]:
# Abort Mission

In [8]:
import pandas as pd
from google.colab import files
file = files.upload()
X_train = pd.read_csv("xtrain.csv", header=None)
Y_train = pd.read_csv("ytrain.csv", header=None)
X_test = pd.read_csv("xtest.csv", header=None)
Y_test = pd.read_csv("ytest.csv", header=None)

Saving xtest.csv to xtest.csv
Saving xtrain.csv to xtrain.csv
Saving ytest.csv to ytest.csv
Saving ytrain.csv to ytrain.csv


Now I'm going to use Keras to define a Neural network that will be trained off of this data. This Neural Network can then be used to predict duture values for breast cancer presence

In [9]:
from keras.models import Sequential
from keras.layers import Dense
classifier = Sequential() # Initializing ANN

classifier.add(Dense(units=16, activation = 'relu', input_dim = 30))
classifier.add(Dense(units=8, activation = 'relu'))
classifier.add(Dense(units=6, activation = 'relu'))
classifier.add(Dense(units=1, activation = 'sigmoid'))

Using TensorFlow backend.


Once the Neural Network is defined, I now have to specify the optimizer and the loss function


In [0]:
classifier.compile(optimizer='rmsprop', loss='binary_crossentropy')

You now train the neural network using Classifier.fit, passing it the training data - i.e. for this set of X, this is what the Y should look like. The NN will then spot the patterns in the data and build a neural network that could replicate that.

In [17]:
  classifier.fit(X_train, Y_train, batch_size=1, epochs=100)

Epoch 1/100
455/455 [==============================] - 0s 1ms/step - loss: 0.0304
Epoch 2/100
455/455 [==============================] - 0s 1ms/step - loss: 0.0376
Epoch 3/100
455/455 [==============================] - 0s 1ms/step - loss: 0.0357
Epoch 4/100
455/455 [==============================] - 0s 1ms/step - loss: 0.0370
Epoch 5/100
455/455 [==============================] - 0s 1ms/step - loss: 0.0326
Epoch 6/100
455/455 [==============================] - 1s 1ms/step - loss: 0.0390
Epoch 7/100
455/455 [==============================] - 0s 1ms/step - loss: 0.0323
Epoch 8/100
455/455 [==============================] - 0s 1ms/step - loss: 0.0241
Epoch 9/100
455/455 [==============================] - 0s 1ms/step - loss: 0.0441
Epoch 10/100
455/455 [==============================] - 1s 1ms/step - loss: 0.0297
Epoch 11/100
455/455 [==============================] - 0s 1ms/step - loss: 0.0264
Epoch 12/100
455/455 [==============================] - 1s 1ms/step - loss: 0.0405
Epoch 13/100


To predict new values, the Neural Network uses classifier.predict. I'm going to pass it the test values for X (which the Neural Network hasn't previously seen ) and it will give me back a set of predictions. These predictions will be probabilities, so I will clean them up by saying that if they'regreater than 0.5, I'll make them 1, else I'll make them 0.

In [0]:
Y_pred = classifier.predict(X_test)
Y_pred = [ 1 if y>=0.5 else 0 for y in Y_pred]

Now we can loop through the set of predictions for the test set (called Y_pred) and the actual values for the test set (called Y_test) and see how alike they are  -- if they are the same, I';; increment 'correct', otherwise I'll increment 'wrong.'

You'll see the result is .... 97 % accurate? We'll see.

In [16]:
total = 0
correct = 0
wrong = 0
for i in range(len(Y_pred)):
  total=total+1
  if(Y_test.at[i,0] == Y_pred[i]):
    correct += 1
  else:
    wrong += 1

print("Total "+ str(total))
print("Correct " + str(correct))
print("Wrong "+ str(wrong))

Total 114
Correct112
Wrong 2
